In [1]:
from transformers import SegformerModel, SegformerConfig, SegformerForSemanticSegmentation
import torch, json

torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
from src.models.Model_TransUNet import TransUNet
from src.models.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg

def create_model(ssize):
    vit_name = 'R50-ViT-B_16'
    config_vit = CONFIGS_ViT_seg[vit_name]
    config_vit.n_classes = 1
    config_vit.n_skip = 3
    img_size = ssize
    vit_patches_size = 16
    if vit_name.find('R50') != -1:
        config_vit.patches.grid = (int(img_size / vit_patches_size), int(img_size / vit_patches_size))
    model = TransUNet(config_vit, img_size=img_size, num_classes=config_vit.n_classes).cuda()
    return model

In [3]:
def measure_vram(ssize):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats("cuda")
    model = create_model(ssize)
    model(torch.rand(size=(1, 3, ssize, ssize), device='cuda'))
    mem_allocation = torch.cuda.max_memory_allocated("cuda")
    mem_allocation_mb = mem_allocation/ 1024**2
    print("Max VRAM:", mem_allocation_mb, "MB")
    return mem_allocation_mb

In [6]:
num_pixel_to_measurements = {}
for mult in [20, 40, 80, 120][::-1]:
    size = mult*16
    num_pixel = size * size
    num_pixel_to_measurements[num_pixel] = []
    for _ in range(3):
        num_pixel_to_measurements[num_pixel].append(measure_vram(size))

json.dump(num_pixel_to_measurements, open("test_transunet.json", "w"), indent=4)


Max VRAM: 22310.40625 MB
Max VRAM: 22310.40625 MB
Max VRAM: 22310.40625 MB
Max VRAM: 6528.34375 MB
Max VRAM: 6528.34375 MB
Max VRAM: 6528.34375 MB
Max VRAM: 2688.28125 MB
Max VRAM: 2688.28125 MB
Max VRAM: 2688.28125 MB
Max VRAM: 2418.97119140625 MB
Max VRAM: 2418.97119140625 MB
Max VRAM: 2418.97119140625 MB


In [8]:

model = create_model(320)
num_parameters = sum(p.numel() for p in model.parameters())
print("Number of parameters:", num_parameters)

Number of parameters: 105432593
